In [1]:
import pandas as pd
from pyproj import Transformer

In [2]:
df = pd.read_csv("../data/xy_mapping/raw/GLIDE Junctions (14042025).csv")
df.head(3)

,FID,Site_ID,Ident,Ident_Len,Juction_Desc,OBU_Displa,Char_Count,Is_PC,Traffic_Light.Junction_X,Traffic_Light.Junction_Y
0,0,101,*0355 KEPPEL RD/SHENTON WY/ECP,30,Keppel Rd/Shenton Wy/ECP,KEPPEL RD/SHENTON WY/ECP,24,False,29359,28349
1,1,102,*#0056 KEPPEL RD/ANSON RD,25,Keppel Rd/Anson Rd,KEPPEL RD/ANSON RD,18,False,29169,28337
2,2,103,*#0161 KEPPEL RD/CANTONMENT RD,30,Keppel Rd/Cantonment Rd,KEPPEL RD/CANTONMENT RD,23,False,28955,28347


In [3]:
transformer = Transformer.from_crs("EPSG:3414", "EPSG:4326", always_xy=True)

def convert_coords(row):
    x, y = row["Traffic_Light.Junction_X"], row["Traffic_Light.Junction_Y"]

    if x == 0 or y == 0:
        return pd.Series([0, 0], index=["lat", "lon"])

    lon, lat = transformer.transform(x, y)
    return pd.Series([lat, lon], index=["lat", "lon"])

df[["lat", "lon"]] = df.apply(convert_coords, axis=1)
df = df[["Site_ID", "lat", "lon"]]

df


,Site_ID,lat,lon
0,101,1.272653,103.845530
1,102,1.272544,103.843822
2,103,1.272635,103.841900
3,104,1.272571,103.839923
4,105,1.274055,103.841378
...,...,...,...
2815,6200,1.308428,103.923955
2816,6226,1.303265,103.896819
2817,6310,1.306846,103.883736
2818,6454,1.309505,103.899047


In [42]:
from datetime import datetime
today_str = datetime.today().strftime("%Y%m%d")
output_path = f"../data/xy_mapping/processed/xy_mapping_{today_str}.csv"
df.to_csv(output_path, index=False)